# Первая модель 9 декабря 2025

## 1. Импорт библиотек и загрузка данных
Раздел «Imports & Data Loading»:  
Импорты  
Загрузка train 
Быстрый .head()  
Проверка формы датасета  
Краткие описания колонок (комментариями)

### Импорты

In [2]:
import os

import pandas as  pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

### Загрузка train и Быстрый .head()

In [3]:
data_path = '../../Data/Kaggle/titanic/'

files = os.listdir(data_path)
for i in range(len(files)):
    print(f'file {i} - {files[i]}')

file 0 - test.csv
file 1 - train.csv
file 2 - gender_submission.csv


In [4]:
df_train_row = pd.read_csv(f'{data_path}{files[1]}')
df_train_row.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Краткие описания колонок (комментариями)

PassengerId — уникальный идентификатор пассажира, просто индекс.  
Survived — целевая переменная: 0 = погиб, 1 = выжил.  
Pclass — класс обслуживания: 1-й (дорогой), 2-й, 3-й (дешёвый).  
Name — имя (можно извлечь звание/титул: Mr, Miss, Mrs, etc.).  
Sex — пол (ключевой фактор выживания).  
Age — возраст (много пропусков → важна обработка).  
SibSp — количество родственников по боковой линии (братья, сёстры, супруги).  
Parch — количество родителей и детей.  
Ticket — номер билета (можно делать группировки).  
Fare — стоимость билета (важно для богатых → выше шанс выжить).  
Cabin — каюта (почти всё пропущено, но буква палубы полезна).  
Embarked — порт отправления: S, C, Q.  
  
  
Так какбудет использоваться логистическая регресссия, то порядковая переменная будет обрабатываться как номинальная
До заполнения пропусков нужно обработать выбросы

## 2. Анализ данных и обработка признаков

Общие статитики по датасету  
Анализ численных признаков  
Обработка числовых признаков  
Анализ нечисловых признаков  
Обработка нечисловых признаков  
Создание новых признкаов

### Общие статитики по датасету

In [5]:
#Проверка на пропуски

df_train_row.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# Статистики

display(df_train_row.describe())
display(df_train_row.describe(include=['object']))

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,G6,S
freq,1,577,7,4,644


### Распределение переменных по типу 

In [7]:
feature_id = 'PassengerId'
target_name = 'Survived'
list_num_ftrs = [	
     'Age'
    ,'SibSp'
    ,'Parch'
    ,'Fare'
]
list_not_num_ftrs = [
     'Pclass'
    ,'Name'
    ,'Sex'
    ,'Ticket'
    ,'Cabin'
    ,'Embarked'

]

if set(df_train_row.columns) == set(list_num_ftrs + list_not_num_ftrs + [feature_id, target_name]):
    print('Все переменные распределены')
else:
    print('Переменные не распределены')

Все переменные распределены


In [8]:
display(df_train_row[list_num_ftrs].info())
display(df_train_row[list_not_num_ftrs].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     714 non-null    float64
 1   SibSp   891 non-null    int64  
 2   Parch   891 non-null    int64  
 3   Fare    891 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 28.0 KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pclass    891 non-null    int64 
 1   Name      891 non-null    object
 2   Sex       891 non-null    object
 3   Ticket    891 non-null    object
 4   Cabin     204 non-null    object
 5   Embarked  889 non-null    object
dtypes: int64(1), object(5)
memory usage: 41.9+ KB


None